In [23]:
from pathlib import Path
import numpy as np

In [2]:
from PIL import Image
import tensorflow as tf

In [7]:
FILE_PATH = Path('data/raw/Flickr2K')
print(FILE_PATH)

data/raw/Flickr2K


In [4]:
HR_SIZE = 128
SCALE = 2
BATCH_SIZE = 8

In [16]:
files = []
files.extend(FILE_PATH.rglob('*.jpg'))
files.extend(FILE_PATH.rglob('*.png'))
files = sorted(files)

In [17]:
print (len(files))

2650


In [18]:
print(files)

[PosixPath('data/raw/Flickr2K/000001.png'), PosixPath('data/raw/Flickr2K/000002.png'), PosixPath('data/raw/Flickr2K/000003.png'), PosixPath('data/raw/Flickr2K/000004.png'), PosixPath('data/raw/Flickr2K/000005.png'), PosixPath('data/raw/Flickr2K/000006.png'), PosixPath('data/raw/Flickr2K/000007.png'), PosixPath('data/raw/Flickr2K/000008.png'), PosixPath('data/raw/Flickr2K/000009.png'), PosixPath('data/raw/Flickr2K/000010.png'), PosixPath('data/raw/Flickr2K/000011.png'), PosixPath('data/raw/Flickr2K/000012.png'), PosixPath('data/raw/Flickr2K/000013.png'), PosixPath('data/raw/Flickr2K/000014.png'), PosixPath('data/raw/Flickr2K/000015.png'), PosixPath('data/raw/Flickr2K/000016.png'), PosixPath('data/raw/Flickr2K/000017.png'), PosixPath('data/raw/Flickr2K/000018.png'), PosixPath('data/raw/Flickr2K/000019.png'), PosixPath('data/raw/Flickr2K/000020.png'), PosixPath('data/raw/Flickr2K/000021.png'), PosixPath('data/raw/Flickr2K/000022.png'), PosixPath('data/raw/Flickr2K/000023.png'), PosixPath(

In [20]:
print(len(files))
print(files[:3])
print(type(files[0]))

2650
[PosixPath('data/raw/Flickr2K/000001.png'), PosixPath('data/raw/Flickr2K/000002.png'), PosixPath('data/raw/Flickr2K/000003.png')]
<class 'pathlib.PosixPath'>


In [25]:
np.random.seed(7)

In [31]:
np.random.shuffle(files)

In [32]:
print(files[:10])

[PosixPath('data/raw/Flickr2K/001583.png'), PosixPath('data/raw/Flickr2K/001605.png'), PosixPath('data/raw/Flickr2K/000279.png'), PosixPath('data/raw/Flickr2K/002041.png'), PosixPath('data/raw/Flickr2K/000187.png'), PosixPath('data/raw/Flickr2K/001694.png'), PosixPath('data/raw/Flickr2K/002460.png'), PosixPath('data/raw/Flickr2K/002077.png'), PosixPath('data/raw/Flickr2K/001197.png'), PosixPath('data/raw/Flickr2K/000340.png')]


In [34]:
train_files = files[:2120]

In [36]:
len(train_files)

2120

In [40]:
val_files = files[2120:2120+265]

In [41]:
len(val_files)

265

In [43]:
test_files = files[2120+265:]

In [44]:
len(test_files)

265